This Notebook was made in Google Colab. So it don't have the virtual environment libraries, local files path and loggings.
Libraries has been installed externally and file paths are given by inputs.

In [1]:
!apt-get install inxi &> /dev/null
!inxi -C

CPU:
  Topology: Single Core model: Intel Xeon bits: 64 type: MT 
  L2 cache: 45.0 MiB 
  Speed: 2300 MHz min/max: N/A Core speeds (MHz): 1: 2300 2: 2300 


In [2]:
!lscpu |grep 'Model name'

Model name:                      Intel(R) Xeon(R) CPU @ 2.30GHz


In [3]:
!nvidia-smi

Mon Jul 17 11:56:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.4/605.4 kB 11.2 MB/s eta 0:00:00


In [5]:
!pip install python-box

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 36.5 MB/s eta 0:00:00


In [6]:
from box import ConfigBox
from box.exceptions import BoxValueError
from dataclasses import dataclass
from pathlib import Path
import yaml
import os

In [7]:
## helper functions

def read_yaml(path_to_yaml: Path) -> ConfigBox:
    """reads yaml file and returns
    Args:
        path_to_yaml (str): path like input
    Raises:
        ValueError: if yaml file is empty
        e: empty file
    Returns:
        ConfigBox: ConfigBox type
    """
    try:
        with open(path_to_yaml) as yaml_file:
            content = yaml.safe_load(yaml_file)
            return ConfigBox(content)
    except BoxValueError:
        raise ValueError("yaml file is empty")
    except Exception as e:
        raise e

In [8]:
@dataclass(frozen = True)
class ModelEvaluateConfig:
  best_model: Path
  data_file: Path
  batch_size: int
  image_size: int
  confidence: float

In [9]:
CONFIG_FILE_PATH = "/content/drive/MyDrive/Object_Detection_Projects/config.yaml"
PARAMS_FILE_PATH = "/content/drive/MyDrive/Object_Detection_Projects/params.yaml"

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_model_evaluate_config(self) -> ModelEvaluateConfig:
      config = self.config.model_evaluation
      params = self.params.MODEL_EVALUATE_PARAMS
      model_evaluate_config = ModelEvaluateConfig(
          best_model = config.best_model,
          data_file = config.data_file_path,
          batch_size= params.BATCH_SIZE,
          image_size = params.IMAGE_SIZE,
          confidence = params.CONFIDENCE
      )


      return model_evaluate_config

In [11]:
from ultralytics import YOLO

In [12]:
class EvaluateModel:
  def __init__(self, config: ModelEvaluateConfig):
    self.config = config

  def evaluate_model(self):
    try:
      model = YOLO(self.config.best_model)
      model.val(
          data = self.config.data_file,
          imgsz = self.config.image_size,
          batch = self.config.batch_size,
          conf = self.config.confidence,
          save_json = True
      )
    except Exception as e:
      raise e


In [16]:
try:
  config = ConfigurationManager()
  model_evaluate_config = config.get_model_evaluate_config()
  model_evaluation = EvaluateModel(model_evaluate_config)
  model_evaluation.evaluate_model()
except Exception as e:
  raise e

Ultralytics YOLOv8.0.136 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 43612005 parameters, 0 gradients
100%|██████████| 755k/755k [00:00<00:00, 15.8MB/s]
val: Scanning /content/drive/MyDrive/Object_Detection_Projects/split_data/val_data/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:19<00:00,  1.95it/s]
                   all        300       2023      0.962      0.883      0.934      0.737
                  open        300        398      0.966      0.917      0.951      0.699
                 short        300        308       0.92      0.821      0.896      0.632
              mousebit        300        414      0.989      0.891      0.944      0.745
                  spur        300        312      0.996      0.897      0.948      0.729
                copper        3